In [25]:
import os
import re
import csv
import sys
import csv
import random
import ast

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [26]:
# load data
recs = []
with open("all-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
len(recs)
recs[0]

{'From': 'Kuwait',
 'action': "['establish']",
 'decision': 'support',
 'id': '0',
 'institutions': '[]',
 'notes': '',
 'text': '69.1 The establishment of an international round table to discuss the interrelation between security and fundamental freedoms (Kuwait). );',
 'theme': '[]',
 'to': 'Algeria',
 'year': '2008'}

## 1. Clean

In [4]:
# check for broken csv rows
for i in recs:
    if not i['year'].startswith('2'):
        print i['text']

In [5]:
""" Assign ID
for i in recs:
    i['id'] = recs.index(i)
"""

" Assign ID\nfor i in recs:\n    i['id'] = recs.index(i)\n"

In [33]:
# ordering
"""
numbs = []
for i in recs[:500]:
    n = i['text'].split()[0]
    n = n.split('.')
    i['n1'] = n[0]
    i['n2'] = n[1]
    if len(n) == 3:
        i['n3'] = n[2]
    else:
        i['n3'] = 0
"""

## 2. Sample

In [249]:
# sample
sample = random.sample(range(1, len(recs)), 100)

recs_sample = []
for i in recs:
    if int(i['id']) in sample:
        recs_sample.append(i)

In [211]:
# delete data we don't want to include
for i in recs_sample:
    del i['institutions']
    del i['theme']
    del i['action']

In [212]:
# check
recs_sample[0]

{'From': 'Portugal',
 'decision': 'consider',
 'id': '441',
 'notes': '',
 'text': '57.2.6 to ensure that laws combating FGM and forced marriages are compatible with its international human rights obligations and that resources are allocated, including through multilateral international cooperation, to fully implement such laws (Portugal);',
 'to': 'Mali',
 'year': '2008'}

In [10]:
#writing column headings
keys = recs_sample[0].keys()
keys

['From', 'text', 'decision', 'to', 'notes', 'year', 'id']

In [11]:
#writing the rest
with open('sample-data.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(recs_sample)

In [22]:
"""
# try this if it doesn't work
probs = []
for i in recs:
    all_keys = i.keys()
    all_keys = [key for key in all_keys if key not in keys]
    if all_keys != []:
        probs.append(recs.index(i))

for i in probs:
    del recs[i][None]
"""

KeyboardInterrupt: 

## 3. Test

In [27]:
# load data
erin = []
with open("testing/uprcodingerin.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                erin.append(row)
            
erin[1]

{'From': 'Tunisia',
 'action': "['continue']",
 'decision': 'consider',
 'id': '690',
 'institutions': '[]',
 'notes about coding': '',
 'sentiment': '3',
 'text': '58.14 To continue to work on improving the situation of children\x90s rights (Algeria, Tunisia);',
 'theme': "['children']",
 'to': 'Romania',
 'year': '2008'}

In [28]:
# making literal lists
for i in erin:
    i['theme'] = ast.literal_eval(i['theme'])
    i['institutions'] = ast.literal_eval(i['institutions'])
    i['action'] = ast.literal_eval(i['action'])

erin[1]

{'From': 'Tunisia',
 'action': ['continue'],
 'decision': 'consider',
 'id': '690',
 'institutions': [],
 'notes about coding': '',
 'sentiment': '3',
 'text': '58.14 To continue to work on improving the situation of children\x90s rights (Algeria, Tunisia);',
 'theme': ['children'],
 'to': 'Romania',
 'year': '2008'}

In [29]:
# get ids
ids = [rec['id'] for rec in erin]

In [30]:
# get original sample
sample = []
for i in recs:
    if i['id'] in ids:
        sample.append(i)
sample[0]  

{'From': 'Mexico',
 'action': '[]',
 'decision': 'support',
 'id': '92',
 'institutions': '[]',
 'notes': '',
 'text': '83.8 Enhance access to justice as well as improve the judicial system (Mexico);',
 'theme': "['judiciary']",
 'to': 'Brazil',
 'year': '2008'}

In [31]:
for i in sample:
    i['theme'] = ast.literal_eval(i['theme'])
    i['institutions'] = ast.literal_eval(i['institutions'])
    i['action'] = ast.literal_eval(i['action'])

sample[1]

{'From': 'Tunisia',
 'action': ['continue'],
 'decision': 'consider',
 'id': '690',
 'institutions': [],
 'notes': '',
 'text': '58.14 To continue to work on improving the situation of children\xcds rights (Algeria, Tunisia);',
 'theme': ['children'],
 'to': 'Romania',
 'year': '2008'}

In [32]:
# zip
matching = []
for i in range(100):
    dic = erin[i].copy()
    
    # Theme
    dic['theme-erin'] = erin[i]['theme']
    dic['theme-rochelle'] = sample[i]['theme']
    del dic['theme']
    
    if set(dic['theme-erin']) == set(dic['theme-rochelle']):
        dic['theme-match'] = True
    elif set(dic['theme-erin']).intersection(dic['theme-rochelle']):
        dic['theme-match'] = "Partial"
    else:
        dic['theme-match'] = False
    
    # institutions
    dic['institution-erin'] = erin[i]['institutions']
    dic['institution-rochelle'] = sample[i]['institutions']
    del dic['institutions']
    
    if set(dic['institution-erin']) == set(dic['institution-rochelle']):
        dic['institution-match'] = True
    elif set(dic['institution-erin']).intersection(dic['institution-rochelle']):
        dic['institution-match'] = "Partial"
    else:
        dic['institution-match'] = False
    
    # actions
    dic['action-erin'] = erin[i]['action']
    dic['action-rochelle'] = sample[i]['action']
    del dic['action']
    
    if set(dic['action-erin']) == set(dic['action-rochelle']):
        dic['action-match'] = True
    elif set(dic['action-erin']).intersection(dic['action-rochelle']):
        dic['action-match'] = "Partial"
    else:
        dic['action-match'] = False
    
    matching.append(dic)

In [33]:
# make lists of matches, etc
theme_match = [i for i in matching if i['theme-match'] == True]
theme_partial = [i for i in matching if i['theme-match'] == "Partial"]
theme_false = [i for i in matching if i['theme-match'] == False]

institution_match = [i for i in matching if i['institution-match'] == True]
institution_partial = [i for i in matching if i['institution-match'] == "Partial"]
institution_false = [i for i in matching if i['institution-match'] == False]

action_match = [i for i in matching if i['action-match'] == True]
action_partial = [i for i in matching if i['action-match'] == "Partial"]
action_false = [i for i in matching if i['action-match'] == False]

In [34]:
print "Theme" + "\n"
print "Match: " + str(len(theme_match))
print "Partial: " + str(len(theme_partial))
print "False: " + str(len(theme_false))
print "\n"

print "Institutions" + "\n"
print "Match: " + str(len(institution_match))
print "Partial: " + str(len(institution_partial))
print "False: " + str(len(institution_false))
print "\n"

print "Action" + "\n"
print "Match: " + str(len(action_match))
print "Partial: " + str(len(action_partial))
print "False: " + str(len(action_false))
print "\n"

Theme

Match: 54
Partial: 29
False: 17


Institutions

Match: 88
Partial: 2
False: 10


Action

Match: 80
Partial: 4
False: 16




## Turn to Dummy SpreadSheet

In [35]:
institutions = ['core', 'CRPD', 'ICCPR', 'eu-vaw', 'geneva', 'ICC', 'ILO', 'ICESR', 'achr', 'CAT', 'CRC', 'echr', '1954-stateless', 'traffick', 'CTOC', 'ICMW', 'ICERD', 'eu-child', 'sp-pr', 'PP', 'kampala', '1951refugees', 'pol-women', 'CPED', 'uncac', 'ottawa', '1961-stateless', 'CEDAW', 'unesco-ed', 'hague', 'genocide', 'HRC', 'unesco-expr', 'int-cord', 'frame-minorities', 'ind169', 'OP']

In [36]:
actions = ['accede', 'ratify', 'take-measures', 'sign', 'report', 'continue', 'withdraw', 'harmonize', 'party', 'implement', 'establish']

In [37]:
theme = ['death-pen', 'infrastructure ', 'intelligence', 'prisoners', 'elderly', 'culture', 'education', 'children', 'civil-so', 'disappearances', 'police', 'hiv', 'privacy', 'discrimination', 'impunity', 'governance', 'labor', 'environment', 'religion', 'foreign-movement', 'health', 'speech', 'domest-movement', 'terrorism', 'paramilitary', 'corruption', 'sex-violence', 'civil-conflict', 'race', 'torture', 'food', 'extra-judicial', 'judiciary', 'indigenous', 'lgbt', 'self-determ', 'poverty', 'ethnic', 'women', 'land', 'reproductive', 'post-conflict', 'disabilities', 'migrants', 'pol-imprisonment', 'socialism', 'trafficking', 'military', 'property']

In [38]:
erin[0].keys()

['From',
 'sentiment',
 'text',
 'decision',
 'id',
 'to',
 'theme',
 'year',
 'action',
 'notes about coding',
 'institutions']

In [39]:
for i in sample:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0 
    
    del i['to']
    del i['From']
    del i['text']
    del i['notes']
    del i['action']
    del i['theme']
    del i['decision']
    del i['institutions']
    del i['year']

In [40]:
for i in erin:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0  
            
    del i['to']
    del i['From']
    del i['text']
    del i['year']
    del i['institutions']
    del i['action']
    del i['theme']
    del i['decision']
    del i['notes about coding']
    del i['sentiment']

In [41]:
## writing sample

#writing column headings
keys = sample[0].keys()

#writing the rest
with open('testing/sample-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sample)

In [42]:
## writing erin

#writing column headings
keys = erin[0].keys()

#writing the rest
with open('testing/erin-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(erin)